# KBO 야구 특화 챗봇 

이 노트북은 논문 "A Chatbot for Football Analytics" 아키텍처를 KBO 야구에 적용한 챗봇의 전체 파이프라인을 실행합니다.

## 주요 구성 요소
1. **데이터 적재 (Ingest)**: JSON → ChromaDB 벡터 저장소
2. **쿼리 분류 (Classifier)**: general / season_analysis / match_analysis
3. **하이브리드 검색 (Retriever)**: Semantic + BM25
4. **응답 생성 (Chain)**: LLM 기반 분석
5. **에이전트 (Agent)**: Function Calling 지원

---

## 1. 환경 설정 및 라이브러리 로드

환경 변수를 `.env` 파일에서 로드하고, API 키가 안전하게 관리되는지 확인합니다.

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# 프로젝트 경로 설정
PROJECT_ROOT = Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

# API 키 확인 (값은 출력하지 않음 - 보안)
api_key = os.getenv("OPENAI_API_KEY")
print(f"✅ 프로젝트 루트: {PROJECT_ROOT}")
print(f"✅ OPENAI_API_KEY 로드: {'성공 ✓' if api_key else '실패 ✗ (.env 파일 확인 필요)'}")

if not api_key:
    print("\n⚠️ .env 파일에 OPENAI_API_KEY를 설정하세요:")
    print("   OPENAI_API_KEY=sk-your-api-key-here")

## 2. 프로젝트 파일 구조 분석

현재 프로젝트의 파일 구조를 트리 형태로 출력합니다.

In [ ]:
def print_tree(path: Path, prefix: str = "", ignore: set = None):
    """디렉토리 구조를 트리 형태로 출력"""
    if ignore is None:
        ignore = {'.git', '__pycache__', '.venv', 'node_modules', '.vscode', 'chroma_db'}
    
    items = sorted(path.iterdir(), key=lambda x: (x.is_file(), x.name))
    items = [i for i in items if i.name not in ignore]
    
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{item.name}")
        
        if item.is_dir():
            next_prefix = "    " if is_last else "│   "
            print_tree(item, prefix + next_prefix, ignore)

print("📁 프로젝트 구조:\n")
print_tree(PROJECT_ROOT)

## 3. 핵심 모듈 임포트

`src/` 디렉토리의 핵심 모듈들을 임포트합니다.

In [ ]:
# 핵심 모듈 임포트
from src.config import (
    CHROMA_DB_DIR, COLLECTION_NAME, EMBEDDING_MODEL,
    SEASON_DATA_DIR, MATCH_DATA_DIR, LLM_MODEL, RETRIEVAL_TOP_K
)

print("📦 설정 정보:")
print(f"   - 임베딩 모델: {EMBEDDING_MODEL}")
print(f"   - LLM 모델: {LLM_MODEL}")
print(f"   - ChromaDB 경로: {CHROMA_DB_DIR}")
print(f"   - 컬렉션 이름: {COLLECTION_NAME}")
print(f"   - 시즌 데이터: {SEASON_DATA_DIR}")
print(f"   - 경기 데이터: {MATCH_DATA_DIR}")
print(f"   - Top-K 검색: {RETRIEVAL_TOP_K}")

## 4. 데이터 적재 (Ingest)

JSON 데이터를 ChromaDB 벡터 저장소에 적재합니다.

### 핵심 전략 (논문 Section 4.2)
- JSON을 그대로 임베딩하지 않음
- **서술형 문장(Descriptive Sentence)** 을 생성하여 임베딩
- 원본 데이터는 metadata에 저장

In [ ]:
from src.ingest import ingest_all_data, initialize_vector_store

# 데이터 적재 실행
# clear_existing=True: 기존 DB 삭제 후 재적재
result = ingest_all_data(clear_existing=True)

if result:
    doc_count = result._collection.count()
    print(f"\n🎉 벡터 스토어 준비 완료! 문서 수: {doc_count}")

## 5. 쿼리 분류 테스트 (Classifier)

사용자 쿼리를 `general`, `season_analysis`, `match_analysis` 중 하나로 분류합니다.

In [ ]:
from src.classifier import classify_query

# 테스트 쿼리들
test_queries = [
    "야구 규칙 설명해줘",              # general
    "한화 이글스 올시즌 성적 분석해줘",   # season_analysis
    "어제 한화 LG 경기 어땠어?",        # match_analysis
]

print("🔍 쿼리 분류 테스트:\n")
for query in test_queries:
    result = classify_query(query)
    print(f"쿼리: {query}")
    print(f"  → 분류: {result.query_type}")
    print(f"  → 팀: {result.teams}")
    print(f"  → 신뢰도: {result.confidence:.0%}")
    print()

## 6. 하이브리드 검색 테스트 (Retriever)

Semantic Search + BM25를 결합한 하이브리드 검색을 수행합니다.

In [ ]:
from src.retriever import retrieve_for_query

# 검색 테스트
test_query = "한화 이글스 시즌 성적"

print(f"🔎 검색 쿼리: '{test_query}'\n")
results = retrieve_for_query(test_query, top_k=3)

for i, doc in enumerate(results, 1):
    print(f"결과 {i}:")
    print(f"  타입: {doc.metadata.get('type')}")
    print(f"  팀: {doc.metadata.get('teams', doc.metadata.get('team'))}")
    print(f"  내용: {doc.page_content[:100]}...")
    print()

## 7. 전체 파이프라인 실행 (Chain)

분류 → 검색 → 생성의 전체 파이프라인을 실행합니다.

In [ ]:
from src.chain import run_analysis

# 분석 실행
query = "한화 이글스 올시즌 성적 분석해줘"
print(f"💬 질문: {query}\n")

result = run_analysis(query)

print(f"📊 분류: {result.query_type}")
print(f"📊 팀: {result.teams}")
print(f"📊 검색 점수: {result.retrieval_score:.2%}")
print(f"📊 대시보드 필요: {result.needs_dashboard}")
print(f"\n🤖 응답:\n{result.response[:500]}...")

## 8. 대화형 에이전트 (Agent)

Function Calling을 지원하는 대화형 에이전트를 사용합니다.

In [ ]:
from src.agent import chat

# 에이전트 대화 테스트
queries = [
    "안녕! KBO에 대해 알려줘",
    "한화 이글스 올시즌 성적 분석해줘",
]

for q in queries:
    print(f"👤 User: {q}")
    response = chat(q)
    print(f"🤖 Assistant: {response.response[:300]}...")
    if response.dashboard:
        print(f"   📊 대시보드 생성됨!")
    print("-" * 50)

---

## 📝 논문 대비 구현 차이점 및 ChromaDB 구성

### 논문과 동일하게 구현한 부분
| 논문 섹션 | 구현 내용 |
|----------|----------|
| Section 4.2 (Embedding) | `multilingual-e5-large-instruct` 모델 사용, L2 정규화 |
| Section 4.3.1 (Query Classification) | CoT 프롬프팅으로 general/season/match 분류 |
| Section 4.3.2 (Query Normalization) | RapidFuzz QRatio 스코러로 팀명 퍼지 매칭 |
| Section 4.4.1 (Instruct Format) | `"Instruct: ... Query: ..."` 포맷 사용 |
| Section 4.4.2 (Hybrid Retrieval) | Semantic (0.8) + BM25 (0.2) 앙상블 |
| Section 4.5.4 (Function Calling) | 대시보드 생성용 도구 정의 |

### 논문과 다르게 (KBO에 맞게) 수정한 부분

| 항목 | 논문 (축구) | 이 프로젝트 (야구) |
|-----|-----------|------------------|
| **도메인** | 유럽 축구 리그 | KBO 한국 프로야구 |
| **TEAM_MAP** | 축구팀 + 대회 매핑 | 10개 KBO 팀 + 한글 별칭 |
| **데이터 구조** | 리그/경기 JSON | 시즌/경기 JSON (야구 스탯) |
| **KPI** | xG, 패스 등 | 타율, 방어율, OPS 등 |
| **Instruct 프롬프트** | "football dataset" | "baseball dataset" |

### ChromaDB 벡터 저장소 구성

```
data/
└── chroma_db/           ← 논문과 동일하게 로컬 영구 저장소 사용
    ├── chroma.sqlite3   ← 메타데이터 저장
    └── [UUID 폴더들]/   ← 벡터 임베딩 저장
```

- **저장 위치**: `data/chroma_db/` (config.py의 `CHROMA_DB_DIR`)
- **컬렉션 이름**: `kbo_data` (config.py의 `COLLECTION_NAME`)
- **임베딩 차원**: 1024 (multilingual-e5-large-instruct)
- **인덱스 방식**: HNSW (ChromaDB 기본값)

> 💡 논문에서는 PlaymakerAI API에서 데이터를 가져왔지만, 이 프로젝트는 로컬 JSON 파일을 사용합니다.
> 이는 API 없이도 테스트할 수 있도록 한 설계 결정입니다.

In [ ]:
# ChromaDB 상태 확인
from src.ingest import initialize_vector_store

vector_store = initialize_vector_store(documents=None)  # 기존 스토어 로드
collection = vector_store._collection

print("🗄️ ChromaDB 벡터 저장소 정보:")
print(f"   - 저장 경로: {CHROMA_DB_DIR}")
print(f"   - 컬렉션 이름: {collection.name}")
print(f"   - 문서 수: {collection.count()}")
print(f"   - 임베딩 모델: {EMBEDDING_MODEL}")
print(f"   - 임베딩 차원: 1024 (multilingual-e5-large-instruct)")

## 1. Environment Setup and Configuration

Load environment variables and initialize required libraries.

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up project paths
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
CHROMA_DB_DIR = DATA_DIR / "chroma_db"
SRC_DIR = PROJECT_ROOT / "src"

# Add src to path
sys.path.insert(0, str(SRC_DIR))

# Verify paths
print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Directory: {DATA_DIR}")
print(f"Source Directory: {SRC_DIR}")

In [ ]:
# Import required libraries
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Any

# LLM and Vector DB
# import openai
# import chromadb
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import Chroma

# Web scraping
# from selenium import webdriver
# from bs4 import BeautifulSoup
# import requests

# Text processing
# from rapidfuzz import fuzz, process

# Local modules
# from scraper import scrape_player_stats, scrape_matches
# from data_processor import load_raw_json, process_player_data, process_match_data
# from vector_store import initialize_chroma_db, embed_and_store, query_vector_store
# from classifier import classify_query
# from retriever import semantic_search, keyword_search, hybrid_search
# from generator import generate_response, call_function, format_response

print("✓ All imports configured successfully")
print(f"OpenAI API Key configured: {'OPENAI_API_KEY' in os.environ}")

## 2. Data Scraping and Collection

Scrape KBO/Statiz websites for player statistics and match data.

In [ ]:
# Example: Data scraping configuration
# NOTE: Uncomment and implement actual scraping logic when ready

SCRAPING_CONFIG = {
    "seasons": [2024, 2025],
    "targets": {
        "player_stats": "https://www.statiz.co.kr/",
        "match_results": "https://www.statiz.co.kr/game/"
    },
    "selenium_headless": True,
    "request_timeout": 30,
    "retry_count": 3
}

print("Scraping Configuration:")
print(json.dumps(SCRAPING_CONFIG, indent=2, ensure_ascii=False))

In [ ]:
# TODO: Implement actual scraping
# Example function call:
# player_data = scrape_player_stats(season=2024)
# match_data = scrape_matches(season=2024, start_date="2024-03-01", end_date="2024-10-31")
# Save to data/raw/season_2024/

print("[SCRAPING STEP] - Ready to implement scraping logic")

## 3. Data Processing and Normalization

Clean and structure scraped data into JSON format.

In [ ]:
# Example data processing
PROCESSING_CONFIG = {
    "input_format": "json",
    "output_format": "json",
    "standardization": {
        "player_stats": ["name", "team", "position", "avg", "hr", "rbi", "era", "wins"],
        "match_data": ["date", "home_team", "away_team", "home_score", "away_score", "box_score"]
    },
    "validation_rules": {
        "required_fields": True,
        "data_types": True,
        "range_checks": True
    }
}

print("Processing Configuration:")
print(json.dumps(PROCESSING_CONFIG, indent=2, ensure_ascii=False))

In [ ]:
# TODO: Implement data processing
# Example:
# for json_file in RAW_DATA_DIR.glob('season_2024/*.json'):
#     raw_data = load_raw_json(str(json_file))
#     if 'player' in json_file.name:
#         processed = process_player_data(raw_data)
#     else:
#         processed = process_match_data(raw_data)
#     # Save processed data

print("[DATA PROCESSING STEP] - Ready to implement processing logic")

## 4. Vector Store Creation with ChromaDB

Embed processed data and store in ChromaDB.

In [ ]:
# Vector store configuration
VECTOR_STORE_CONFIG = {
    "embedding_model": "text-embedding-ada-002",
    "chunk_size": 500,
    "chunk_overlap": 50,
    "persist_directory": str(CHROMA_DB_DIR),
    "collection_names": [
        "player_statistics",
        "match_records",
        "season_analysis"
    ],
    "similarity_metric": "cosine"
}

print("Vector Store Configuration:")
print(json.dumps(VECTOR_STORE_CONFIG, indent=2, ensure_ascii=False))

In [ ]:
# TODO: Initialize ChromaDB and create embeddings
# Example:
# chroma_client = initialize_chroma_db(persist_dir=str(CHROMA_DB_DIR))
# 
# # Load and embed player statistics
# player_docs = load_documents(DATA_DIR / "raw" / "season_2024")
# embed_and_store(
#     documents=player_docs,
#     collection_name="player_statistics"
# )

print("[VECTOR STORE STEP] - Ready to create embeddings and store in ChromaDB")

## 5. Query Classification System

Classify user queries using LLM-based classification.

In [ ]:
# Query classification configuration
CLASSIFIER_CONFIG = {
    "model": "gpt-3.5-turbo",
    "intent_types": [
        "player_statistics",
        "match_results",
        "season_analysis",
        "team_performance",
        "prediction",
        "comparison",
        "general_question"
    ],
    "temperature": 0.3,
    "max_tokens": 200
}

print("Classifier Configuration:")
print(json.dumps(CLASSIFIER_CONFIG, indent=2, ensure_ascii=False))

In [ ]:
# Example queries for classification testing
test_queries = [
    "2024년 최고 타율을 기록한 선수는 누구입니까?",
    "어제 삼성과 LG의 경기 결과는?",
    "올 시즌 강정호 선수의 홈런 통계를 보여줘",
    "2024 정규시즌과 2025 시작 전 팀의 전력을 비교해줘",
    "다음 주 우리 팀의 승리 확률은?"
]

print("Example Queries for Classification:")
for i, query in enumerate(test_queries, 1):
    print(f"{i}. {query}")

In [ ]:
# TODO: Implement query classification
# Example:
# for query in test_queries:
#     classification = classify_query(query)
#     print(f"Query: {query}")
#     print(f"Classification: {classification}")

print("[CLASSIFICATION STEP] - Ready to classify user queries")

## 6. Hybrid Retrieval Implementation

Combine semantic search and keyword-based retrieval.

In [ ]:
# Retrieval configuration
RETRIEVER_CONFIG = {
    "semantic_search": {
        "model": "text-embedding-ada-002",
        "top_k": 5,
        "similarity_threshold": 0.5
    },
    "keyword_search": {
        "method": "rapidfuzz",
        "matcher": "token_sort_ratio",
        "score_threshold": 70,
        "top_k": 5
    },
    "hybrid_search": {
        "semantic_weight": 0.6,
        "keyword_weight": 0.4,
        "top_k": 10,
        "rerank": True
    }
}

print("Retriever Configuration:")
print(json.dumps(RETRIEVER_CONFIG, indent=2, ensure_ascii=False))

In [ ]:
# TODO: Implement hybrid retrieval
# Example:
# query = "2024년 삼성 라이온즈 선수 통계"
# results = hybrid_search(
#     query=query,
#     top_k=10,
#     alpha=0.6
# )
# 
# for doc in results:
#     print(f"Score: {doc['score']:.3f}")
#     print(f"Content: {doc['content'][:200]}...")

print("[RETRIEVAL STEP] - Ready to perform hybrid search")

## 7. LLM Response Generation with Function Calling

Generate responses using LLM with function calling capabilities.

In [ ]:
# Response generation configuration
GENERATOR_CONFIG = {
    "model": "gpt-3.5-turbo",
    "temperature": 0.7,
    "max_tokens": 1000,
    "functions": [
        {
            "name": "get_player_stats",
            "description": "Get detailed statistics for a specific player",
            "parameters": {
                "type": "object",
                "properties": {
                    "player_name": {"type": "string"},
                    "season": {"type": "integer"}
                },
                "required": ["player_name"]
            }
        },
        {
            "name": "get_team_stats",
            "description": "Get team statistics and standings",
            "parameters": {
                "type": "object",
                "properties": {
                    "team_name": {"type": "string"},
                    "season": {"type": "integer"}
                },
                "required": ["team_name"]
            }
        },
        {
            "name": "get_match_result",
            "description": "Get match result and box score",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string"},
                    "teams": {"type": "array"}
                },
                "required": ["date"]
            }
        }
    ],
    "include_sources": True
}

print("Generator Configuration:")
print(json.dumps({k: v for k, v in GENERATOR_CONFIG.items() if k != 'functions'}, indent=2, ensure_ascii=False))
print(f"\nFunction calling enabled: {len(GENERATOR_CONFIG['functions'])} functions available")

In [ ]:
# TODO: Implement response generation
# Example:
# user_query = "2024년 삼성의 최고 타자는 누구?"
# 
# # Classify query
# classification = classify_query(user_query)
# 
# # Retrieve context
# context_docs = hybrid_search(user_query, top_k=5)
# 
# # Generate response
# response = generate_response(
#     query=user_query,
#     context=context_docs,
#     use_function_calling=True
# )
# 
# # Format final response
# final_response = format_response(response, include_sources=True)
# print(json.dumps(final_response, indent=2, ensure_ascii=False))

print("[GENERATION STEP] - Ready to generate LLM responses")

## 8. End-to-End Pipeline Testing

Test the complete pipeline with sample queries.

In [ ]:
# Complete pipeline test queries
pipeline_test_queries = [
    "2024 시즌 평균 타율 0.300 이상인 선수들의 목록을 보여줘",
    "삼성 라이온즈의 2024년 홈런 통계는?",
    "지난 주말 경기 결과들을 정리해줄래?",
    "2024년 투수 방어율 통계 TOP 5를 알려줘",
    "우리 팀이 이번 시즌 몇 경기를 이겼나요?"
]

print("End-to-End Pipeline Test Queries:")
for i, query in enumerate(pipeline_test_queries, 1):
    print(f"\n{i}. {query}")

In [ ]:
# TODO: Run complete pipeline tests
# def run_complete_pipeline(user_query: str) -> Dict[str, Any]:
#     """Run the complete RAG pipeline for a user query."""
#     
#     # Step 1: Classify query
#     classification = classify_query(user_query)
#     
#     # Step 2: Retrieve relevant documents
#     retrieved_docs = hybrid_search(user_query, top_k=5)
#     
#     # Step 3: Generate response
#     response = generate_response(
#         query=user_query,
#         context=retrieved_docs,
#         use_function_calling=True
#     )
#     
#     return {
#         "query": user_query,
#         "classification": classification,
#         "retrieved_documents": retrieved_docs,
#         "response": response
#     }
# 
# # Run tests
# test_results = []
# for query in pipeline_test_queries:
#     result = run_complete_pipeline(query)
#     test_results.append(result)
#     print(f"\n[TEST] Query: {query}")
#     print(f"Classification: {result['classification']['intent']}")
#     print(f"Response: {result['response'][:200]}...")

print("[PIPELINE TESTING] - Ready to test complete RAG system")

In [ ]:
# Summary and system status
SYSTEM_STATUS = {
    "status": "Configuration Complete",
    "components": {
        "environment_setup": "✓ Completed",
        "data_scraping": "○ Ready to implement",
        "data_processing": "○ Ready to implement",
        "vector_store": "○ Ready to implement",
        "query_classifier": "○ Ready to implement",
        "hybrid_retrieval": "○ Ready to implement",
        "response_generator": "○ Ready to implement",
        "pipeline_testing": "○ Ready to implement"
    },
    "next_steps": [
        "1. Implement scraping logic in src/scraper.py",
        "2. Implement data processing in src/data_processor.py",
        "3. Implement vector store in src/vector_store.py",
        "4. Implement classifier in src/classifier.py",
        "5. Implement retriever in src/retriever.py",
        "6. Implement generator in src/generator.py",
        "7. Execute and test complete pipeline"
    ]
}

print("\n=== SYSTEM STATUS ===")
print(json.dumps(SYSTEM_STATUS, indent=2, ensure_ascii=False))